In [ ]:
import pandas as pd
import FIRM.base.operators.implications as implications
import FIRM.base.operators.tnorms as tnorms
import FIRM.base.fuzzy_data as fuzzy_data
from FIRM.methods.AARFI import AARFI

In [ ]:
impl_operators = [implications.ImplicationsExamples.get_fuzzy_implication(implications.ImplicationsExamples.IGNORE),
                  implications.ImplicationsExamples.get_fuzzy_implication(implications.ImplicationsExamples.LUKASIEWICZ),
                  lambda x, y: implications.ImplicationsExamples.get_fuzzy_implication(implications.ImplicationsExamples.CH10)(x,y,float(0.01)),
                  lambda x, y: implications.ImplicationsExamples.get_fuzzy_implication(implications.ImplicationsExamples.KSS)(x, y, float(-10))]
tnorms_operators = [tnorms.TnormsExamples.get_tnorm(tnorms.TnormsExamples.PRODUCT),
                    tnorms.TnormsExamples.get_tnorm(tnorms.TnormsExamples.LUKASIEWICZ),
                    tnorms.TnormsExamples.get_tnorm(tnorms.TnormsExamples.PRODUCT),
                    lambda x, y: tnorms.TnormsExamples.get_tnorm(tnorms.TnormsExamples.SCHWEIZER_SKLAR)(x, y,float(-10))]

In [ ]:
def process_df(df):
    for col in df.columns:
        unique_values = df[col].dropna().unique()  # Get unique values, excluding NaN
        if len(unique_values) < 15:
            df[col] = df[col].astype('object')  # Convert to boolean
        else:
            df[col] = df[col].astype('float64')  # Convert to categorical for other binary cases
    return df

In [ ]:
datasets = ['abalone.csv','iris.csv','wdbc.csv','magic.csv','vehicle.csv']

In [ ]:
import csv
import pandas as pd
import numpy as np

n_datasets = len(datasets)
n_operators = len(impl_operators)

results = [[0 for _ in range(n_datasets)] for _ in range(n_operators)]

# Open a CSV file to write the results
with open('results.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Operator', 'Dataset', 'Num Rules', 'Fcoverage', 'Fsupport', 'Fconfidence','FWRAcc'])

    for idx_op in range(n_operators):
        print('Operator: ' + str(idx_op))
        I = impl_operators[idx_op]
        T = tnorms_operators[idx_op]
        for idx_dat in range(n_datasets):
            print('Dataset: ' + str(datasets[idx_dat]))
            name_dataset = datasets[idx_dat]
            dataset = process_df(pd.read_csv('../assets/' + name_dataset, sep=','))
            fuzzy_dataset = fuzzy_data.FuzzyDataQuantiles(name_dataset, dataset, 3, ['L', 'M', 'H'])
            rules = AARFI(dataset, fuzzy_dataset, T, I, min_cov=0.3, min_supp=0.3, min_conf=0.8, max_feat=3)
            measures = rules.measures(fuzzy_dataset)
            
            # Extract measures
            num_rules = len(measures['num_features'])
            fcoverage = np.mean(measures['fcoverage'])
            fsupport = np.mean(measures['fsupport'])
            fconfidence = np.mean(measures['fconfidence'])
            fwracc = np.mean(measures['fwracc'])
            
            print('num rules: ' + str(len(measures['num_features'])))
            print('fcoverage: ' + str(np.mean(measures['fcoverage'])))
            print('fsupport: ' + str(np.mean(measures['fsupport'])))
            print('fconfidence: ' + str(np.mean(measures['fconfidence'])))
            print('fwracc: ' + str(np.mean(measures['fwracc'])))
            
            # Write the results to the CSV file
            writer.writerow([idx_op, name_dataset, num_rules, fcoverage, fsupport, fconfidence,fwracc])
            
            # Store the rule list in the results matrix
            results[idx_op][idx_dat] = rules.rule_list

print("Results have been saved to 'results.csv'")

In [ ]:
from FIRM.base.ct_set_fuzzy_rules import SetFuzzyRules

In [ ]:
M = np.ones((n_operators, n_operators))
for i in range(0, n_operators):
    for j in range(i + 1, n_operators):
        diss_perc = []
        for d in range(0, n_datasets):
            rules1 = SetFuzzyRules(results[i][d][:]) 
            rules2 = SetFuzzyRules(results[j][d][:])
            diss_perc = diss_perc + [rules1.jaccard_similarity(rules2)]
        M[i][j] = round(np.mean(diss_perc),3)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Data from the table
data = {
    "(TP,IY)": M[0],
    "(TLK,ILK)": M[1],
    "(TP,ICλH)": M[2],
    "(TSSλ,ISSλ)": M[3]
}

# Row labels
index = ["(TP,IY)", "(TLK,ILK)", "(TP,ICλH)", "(TSSλ,ISSλ)"]

# Create a DataFrame
df = pd.DataFrame(data, index=index)

# Mask the upper triangle to show only non-redundant values
mask = np.triu(np.ones_like(df, dtype=bool), k=1)

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(
    df,
    annot=True,
    cmap="coolwarm",  # Brighter colormap (options: "viridis", "plasma", "magma", "inferno")
    fmt=".2f",  # Display two decimal places
    linewidths=0.5,
    mask=mask,  # Apply the mask to hide the upper triangle
    cbar_kws={"shrink": 0.8}
)

# Show the plot
plt.tight_layout()
plt.show()